In [1]:
import sys  
import os
sys.path.append(os.path.abspath(r"C:\Users\Jil Henry\Documents\Programming\Final Year Project\acne_diagnosis\python"))
sys.path.append(os.path.abspath(r"C:\Users\Jil Henry\Documents\Programming\Final Year Project\acne_diagnosis\python\api"))

In [82]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from data_preprocessing import resize_rescale
import constants as Constants
import data_formatter as df

In [83]:
MODEL = tf.keras.models.load_model('./saved_models/7')

In [100]:
#Getting Dataset from Storage
dataset = tf.keras.utils.image_dataset_from_directory(
    './dataset/test/',
    image_size = (224,224),
    shuffle=False,
)
# tf_images = [imgs.numpy() for imgs,_ in dataset.take(1)][0]
# tf_images = resize_rescale(tf_images)

Found 7 files belonging to 1 classes.


In [44]:
print(f"TF Images Aray Type: {tf_images.shape}")

TF Images Aray Type: (7, 224, 224, 3)


In [99]:
PARENT_FILE_PATH = './dataset/test/acne/'
files = os.listdir(PARENT_FILE_PATH)

images_array = []
predictions = []
for file in files:
    pil_image = tf.keras.utils.load_img(PARENT_FILE_PATH + file,target_size=(224,224))
    image_array = tf.keras.preprocessing.image.img_to_array(pil_image)
    image_array = tf.expand_dims(image_array,0)
#     image_array = resize_rescale(image_array)
#     print(f"Image_Array: {image_array.shape}")
    prediction = MODEL.predict(image_array).tolist()
    predictions.append(prediction[0])
predictions

[[0.7557064890861511, 0.24429355561733246],
 [0.5704280734062195, 0.42957189679145813],
 [0.8196278810501099, 0.18037213385105133],
 [0.8360050320625305, 0.16399504244327545],
 [0.8365836143493652, 0.16341635584831238],
 [0.6122899651527405, 0.3877100348472595],
 [0.7663037180900574, 0.23369628190994263]]

In [46]:
print(f"Images Aray Type: {images_array.shape}")

Images Aray Type: (7, 224, 224, 3)


In [101]:
tf_predictions = []
images_array = []
for images,labels in dataset.take(1):
    for i in range(len(images)):
        img_array = tf.keras.preprocessing.image.img_to_array(images[i].numpy())
        img_array = tf.expand_dims(img_array,0)
#         images_array.append(img_array)
        prediction = MODEL.predict(img_array).tolist()
        tf_predictions.append(prediction[0])
tf_predictions

[[0.7571082711219788, 0.24289175868034363],
 [0.5776153802871704, 0.4223845303058624],
 [0.8202861547470093, 0.17971380054950714],
 [0.8351976871490479, 0.16480228304862976],
 [0.8379772305488586, 0.16202273964881897],
 [0.613267183303833, 0.3867327868938446],
 [0.7675308585166931, 0.2324691265821457]]

In [49]:
predictions = MODEL.predict(tf_images_array)
num_predictions = len(predictions)
print(f"Predictions: {predictions.tolist()}")

ValueError: in user code:

    File "C:\Users\Jil Henry\anaconda3\envs\deeplearning\lib\site-packages\keras\engine\training.py", line 1801, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\Jil Henry\anaconda3\envs\deeplearning\lib\site-packages\keras\engine\training.py", line 1790, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Jil Henry\anaconda3\envs\deeplearning\lib\site-packages\keras\engine\training.py", line 1783, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\Jil Henry\anaconda3\envs\deeplearning\lib\site-packages\keras\engine\training.py", line 1751, in predict_step
        return self(x, training=False)
    File "C:\Users\Jil Henry\anaconda3\envs\deeplearning\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\Jil Henry\anaconda3\envs\deeplearning\l

In [ ]:
print(f"Num of Predictions: {num_predictions}")
if num_predictions < 1:
    print(df.format_result(
        status='No Prediction',
        result_type=Constants.RESULT_TYPE_CONC))

pred_classes_code = np.array([], dtype='int64')
confidence_list_total = np.zeros_like(predictions[0])

if num_predictions == 1:
    #return{"prediction":str(predictions)}
    pred_class_code, confidence_list = df.format_prediction_result(
            predictions[0])
    #print(np.round(confidence_list.tolist()[pred_class_code],2))
    print( df.format_result(
            status='OK',
            result_type=Constants.RESULT_TYPE_CONC,
            pred_class=Constants.CLASS_NAMES[pred_class_code],
            confidence= np.round(confidence_list.tolist()[pred_class_code],2) * 100,
        ))
elif num_predictions > 1:
    for prediction in predictions:
        pred_class_code, confidence_list = df.format_prediction_result(
                prediction)
        pred_classes_code = np.append(pred_classes_code, pred_class_code)
        confidence_list_total += confidence_list

    print(f"Confidence Total = {confidence_list_total}")
    # Getting Number of  Most Occurring Class
    pred_classes_code_mode = np.bincount(pred_classes_code)
    max_num = np.max(pred_classes_code_mode)
    pred_classes_code_mode_num = [x for x in pred_classes_code_mode if x == max_num]
    # Return Results
    if len(pred_classes_code_mode_num) > 1:
        print(confidence_list_total)
        print( df.format_result(
              status='Inconclusive',
              result_type=Constants.RESULT_TYPE_INCONC
           ))
    else:
        index = pred_classes_code_mode.argmax()
        print(f"Predicted Index: {index}")
        confidence = np.round(((confidence_list_total[index] / num_predictions) * 100),2)
        pred_class = Constants.CLASS_NAMES[index]
        print( df.format_result(
                    status='OK',
                    result_type=Constants.RESULT_TYPE_CONC,
                    pred_class=pred_class,
                    confidence=confidence
                ))